In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
data = pd.read_csv("../input/goodreadsbooks/books.csv",error_bad_lines=False)


In [4]:
data.columns

In [42]:
#simplified book table
book = data.drop(['bookID', 'authors', 'isbn', '  num_pages', 'ratings_count', 'text_reviews_count', 'publisher', 'publication_date'], axis = 1)
book = book.rename(columns = {'isbn13': 'book_id'})

In [43]:
book.head()

In [7]:
#check if there is null value, if so we will drop them
book.isnull().sum()

In [8]:
author = data.drop(['bookID', 'title', 'average_rating', 'isbn',
       'language_code', '  num_pages', 'ratings_count', 'text_reviews_count',
       'publication_date', 'publisher'], axis = 1)
author

In [9]:
#split authors 
author_split = pd.concat([author['isbn13'], author["authors"].str.split("/", expand=True)], axis = 1)

In [10]:
miss_vals_count = author_split.isnull().sum()
miss_vals_count

In [11]:
author_split = author_split.iloc[: , :-48]

In [19]:
author_split = author_split.rename(columns = {'isbn13': 'book_id', 0: 'author_1', 1 : 'author_2', 2: 'author_3'})

In [35]:
book_author_1 = author_split.drop(['author_2', 'author_3'], axis = 1)
book_author_2 = author_split.drop(['author_1', 'author_3'], axis = 1)
book_author_3 =author_split.drop(['author_1', 'author_2'], axis = 1)
book_author_1 = book_author_1.rename(columns = {'author_1': 'author'})
book_author_2 = book_author_2.rename(columns = {'author_2': 'author'})
book_author_3 = book_author_3.rename(columns = {'author_3': 'author'})
book_author_2 = book_author_2.dropna()
book_author_3 = book_author_3.dropna()

In [38]:
frames = [book_author_1, book_author_2, book_author_3]
book_author = pd.concat(frames)

In [40]:
#create unique author id using hash 
book_author['author_id'] = book_author[['author']].sum(axis=1).map(hash)

In [47]:
#FINAL TABLES FOR BOOK, AUTHOR AND BOOK_AUTHOR TABLES:
#unique values for table will be handled during mysql db creation
book
author = book_author.drop('book_id', axis = 1)
book_author = book_author.drop('author', axis = 1)

In [48]:
book

In [49]:
author

In [50]:
book_author

In [52]:
book.to_csv('/kaggle/working/book.csv', encoding='utf-8')
author.to_csv('/kaggle/working/author.csv', encoding='utf-8')
book_author.to_csv('/kaggle/working/book_author.csv', encoding= 'utf-8')